In [1]:
import csv
import ujson as json
from memoized import memoized
import xml.etree.ElementTree as et

@memoized
def loadjson(table_name):
  return json.load(open('/lfs/local/0/geza/ml-004_data_export/' + table_name + '.json'))

def get_table_path(table_name):
  csv_path = '/lfs/local/0/geza/ml-004_data_export/'
  return csv_path + table_name + '.csv'

def get_table_data(table_name):
  return csv.DictReader(open(get_table_path(table_name)), doublequote=False, delimiter=',', escapechar='\\')

def print_first_lines(table_name, n=10):
  for x in get_table_data(table_name):
    if n <= 0:
      break
    n -= 1
    print x

In [2]:
invideo_quiz_itemids = set()

for line in get_table_data('quiz_metadata'):
  if line['deleted'] != '0':
    continue
  if line['duration'] != '0':
    continue
  quiz_id = int(line['id'])
  if quiz_id > 144:
    continue
  if line['quiz_type'] != 'video':
    continue
  invideo_quiz_itemids.add(quiz_id)


In [3]:
# only enable invideo quizzes with 1 in-video quiz

#invideo_quiz_itemids = set([2, 3, 4, 5, 6, 7, 9, 10, 11, 25, 26, 27, 28, 29, 30, 31, 32, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129])

invideo_quiz_itemids = set([4, 5, 6, 7, 9, 10, 25, 26, 27, 28, 29, 30, 31, 32, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129])


In [4]:
submission_id_to_quiz_id = {}
submission_id_to_user_id = {}

for line in get_table_data('quiz_submission_metadata'):
  item_id = int(line['item_id'])
  submission_id = int(line['id'])
  user_id = line['session_user_id']
  if item_id not in invideo_quiz_itemids:
    continue
  submission_id_to_user_id[submission_id] = user_id
  submission_id_to_quiz_id[submission_id] = item_id


In [9]:
submission_id_to_answer = {}

exit_loop = False

for key,value in loadjson('kvs_course.quiz.submission'):
  if exit_loop:
    break
  parts = key.split('.')
  submission_id = int(parts[1].split(':')[1])
  if submission_id not in submission_id_to_quiz_id:
    continue
  if submission_id not in submission_id_to_user_id:
    continue
  quiz_id = submission_id_to_quiz_id[submission_id]
  user_id = submission_id_to_user_id[submission_id]
  if quiz_id not in invideo_quiz_itemids:
    continue
  answers = value['answers']
  if len(answers) != 1:
    continue
  answer = answers.values()[0].values()[0]
  submission_id_to_answer[submission_id] = answer
  

In [13]:
#json.dump(submission_id_to_answer, open('/lfs/local/0/geza/submission_id_to_answer.json', 'w'))

In [10]:
#print submission_id_to_answer[2097152]

deac2bdac89df36b0158a36172025dc6


In [11]:
quiz_to_answer_to_count = {}

for submission_id,answer in submission_id_to_answer.iteritems():
  if submission_id not in submission_id_to_quiz_id:
    continue
  if submission_id not in submission_id_to_user_id:
    continue
  quiz_id = submission_id_to_quiz_id[submission_id]
  user_id = submission_id_to_user_id[submission_id]
  if quiz_id not in quiz_to_answer_to_count:
    quiz_to_answer_to_count[quiz_id] = {}
  if answer not in quiz_to_answer_to_count[quiz_id]:
    quiz_to_answer_to_count[quiz_id][answer] = 0
  quiz_to_answer_to_count[quiz_id][answer] += 1

In [17]:
quiz_to_correct_answer = {}

for quiz_id,answer_to_count in quiz_to_answer_to_count.iteritems():
  count_and_answer = [(count, answer) for answer,count in answer_to_count.items()]
  best_answer = sorted(count_and_answer)[len(count_and_answer) - 1][1]
  quiz_to_correct_answer[quiz_id] = best_answer 

In [20]:
submission_id_to_correct = {}

for submission_id,answer in submission_id_to_answer.iteritems():
  if submission_id not in submission_id_to_quiz_id:
    continue
  if submission_id not in submission_id_to_user_id:
    continue
  quiz_id = submission_id_to_quiz_id[submission_id]
  user_id = submission_id_to_user_id[submission_id]
  correct_answer = quiz_to_correct_answer[quiz_id]
  correctness = correct_answer == answer
  submission_id_to_correct[submission_id] = correctness

In [21]:
#json.dump(submission_id_to_correct, open('/lfs/local/0/geza/submission_id_to_correct.json', 'w'))

In [18]:
#json.dump(quiz_to_correct_answer, open('/lfs/local/0/geza/quiz_to_correct_answer.json', 'w'))

In [19]:
#print quiz_to_correct_answer[5]

muEi5gyrg3


{4: {u'cIrwuamv01': 4242, u'hg80Rm7BvA': 1110, u'2gBgxGj2NR': 29526, u'5sSZfbyH6N': 277}, 5: {u'YfU9ilgrvJ': 67, u'CEgT77LLCl': 108, u'muEi5gyrg3': 30563, u'VsZTBP5GWA': 759}, 6: {u'YfU9ilgrvJ': 205, u'CEgT77LLCl': 289, u'muEi5gyrg3': 1277, u'VsZTBP5GWA': 30318}, 7: {u'9JcoJAKXH6': 7832, u'AuddBmY8xZ': 22277, u'wEKo9vutSr': 4383, u'ZE1NZI8S5d': 4816}, 9: {u's2jYX64VJX': 21038, u'J3dZndsMLr': 11605, u'WT9Kka8vqB': 2532, u'ywbT93ti2F': 513}, 10: {u'3NAsEMcLCo': 2478, u'fM2bYlhWSR': 21912, u'dWYsa1K4Gg': 1389, u'6nZkhj8Aue': 2540}, 25: {u'ca5993af294fc39ba6a8ac399fec508e': 33, u'ec27a7d997b80dfc8d5034f6de005e26': 295, u'6d7251327a44f6abcbebcd382842f987': 15, u'9dd92b534b3916b1150b36d66a6abd1f': 11470}, 26: {u'bbf92f895f4d3446977389934a88eb3f': 27, u'db022531ce1c28a4c17c36b9a6bc13ad': 124, u'd8b4298857f0043081302476fe5dd67d': 77, u'46a88833c8fd16152926ce7b8ed50d33': 12054}, 27: {u'89dc6767d288438ac6f0178ff5b0e40c': 20438, u'01b94121b090ca3c8135ce4ae1ae0814': 1548, u'821268b0413f509c4a155b5

In [12]:
'''
submission_id_to_answer = {}

quiz_id_to_max_answers = {}

for x in invideo_quiz_itemids:
  quiz_id_to_max_answers[x] = 0

exit_loop = False

for key,value in loadjson('kvs_course.quiz.submission'):
  if exit_loop:
    break
  parts = key.split('.')
  submission_id = int(parts[1].split(':')[1])
  if submission_id not in submission_id_to_quiz_id:
    continue
  if submission_id not in submission_id_to_user_id:
    continue
  quiz_id = submission_id_to_quiz_id[submission_id]
  user_id = submission_id_to_user_id[submission_id]
  if quiz_id not in invideo_quiz_itemids:
    continue
  answers = value['answers']
  quiz_id_to_max_answers[quiz_id] = max(quiz_id_to_max_answers[quiz_id], len(answers))
'''

In [15]:
#print quiz_id_to_max_answers
'''
quizzes_with_one_answer = set()
for k,v in quiz_id_to_max_answers.items():
  if v == 1:
    quizzes_with_one_answer.add(k)

print quizzes_with_one_answer
'''

set([4, 5, 6, 7, 9, 10, 25, 26, 27, 28, 29, 30, 31, 32, 38, 41, 42, 43, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 63, 64, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 129])


In [7]:
'''
#quiz_2_answer_distributions = {}
#quiz_2_answers_key = set()
quiz_id_to_answers_key = {}

# quiz_id = 21 is an example of one that has two
# quiz_id = 22 is an example of a quiz that has four questions!

for key,value in loadjson('kvs_course.quiz.saved'):
  parts = key.split('.')
  quiz_id = int(parts[1].split(':')[1])
  if quiz_id not in invideo_quiz_itemids:
    continue
  user_id = parts[2].split(':')[1]
  if quiz_id not in quiz_id_to_answers_key:
    quiz_id_to_answers_key[quiz_id] = set()
  for x in value['answers'].keys():
    quiz_id_to_answers_key[quiz_id].add(x)
'''

In [10]:
'''
print [len(x) for x in quiz_id_to_answers_key.values()]
print quiz_id_to_answers_key

quiz_ids_with_just_one_question = set()
for k,v in quiz_id_to_answers_key.items():
  if len(v) == 1:
    quiz_ids_with_just_one_question.add(k)

print quiz_ids_with_just_one_question
print len(quiz_ids_with_just_one_question)
'''

[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0]
{2: set([u'l9nypDpiHq']), 3: set([u'XRH6Bfw1ub']), 4: set([u'DIsCI26ARr']), 5: set([u'MPky4kp4xX']), 6: set([u'0d1iR1YFqK']), 7: set([u'd8Rtc7LCHS']), 9: set([u'qC8kiV3NhH']), 10: set([u'F7CFqgWAOd']), 11: set([u'nzOSLZkEXV']), 21: set([u'3c2b16ed68c0fc61df870ea549d79271', u'57859454cf35d425bd94006afdc9e1fe']), 22: set([u'cf2a66ef4fcbb82846f7d784de4763eb', u'969fd508d737a44cd4a5fe69a583942f', u'cda9dad8fb55efcabea787717102f417']), 23: set([u'00eb0e871e6ce85883cddb9d12d566c0', u'8b4486925af827a18afd8fcaaabbf164']), 24: set([u'8a904036386999c4f1f6846b687276c3', u'c8be7aa3a9ad4ddaa7a89759b749d4e5', u'5b2ba42c35aaac2b01cd8a53ea9527d6', u'e5838670d9efc65d7133f262cd3

In [3]:
#for key,value in loadjson('kvs_course.quiz.submission'):
#  print key
#  print value
#  break

submission.submission_id:1
{u'active': True, u'start_time': 1343006234, u'random_seed': 1343006234, u'saved_time': 1343006294, u'answers': {u'49ec22c8a5bcc3e5be33545a76d30b2a': {u'0': u'e09933a3f9f7d62e5f7ecd977a4b4798'}}}


In [ ]:

#1381764882